In [1]:
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import pandas as pd
import os
import pickle
import time
import re

/home/omar/Desktop/Logatta/SearchEngine/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli')

In [3]:
# import data
df = pd.read_csv('myDatabase.facebook_posts.csv')

In [4]:
df.isnull().sum()


_id              0
Post_Content    22
dtype: int64

In [5]:
# drop rows with null values
df = df.dropna()

In [6]:
df.shape

(21134, 2)

In [7]:
ids = df['_id'].tolist()
names = df['Post_Content'].tolist()

In [8]:
# Embed the dataset 
# name_embeddings = model.encode(names, convert_to_tensor=True)
name_embeddings = torch.load('name_embeddings_facebook.pt')

In [9]:
name_embeddings.shape

torch.Size([21134, 768])

In [10]:
def semantic_search(query, embeddings, texts, ids, top_k=100):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)
  
    # Compute cosine similarity between the query and texts
    cos_similarities = util.cos_sim(query_embedding, embeddings)
  
    # Sort the results by cosine similarity in descending order
    sorted_indices = cos_similarities.argsort(descending=True)
  
    # Return the top-k most similar texts and their corresponding indices
    top_k_indices = sorted_indices[0][:top_k]
    return [(texts[i], ids[i]) for i in top_k_indices]

In [12]:
query = input('Enter your query: ')
results = semantic_search(query, name_embeddings, names, ids, top_k=100)
for name, idx in results:
    print(f'name: {name} -  id: {idx}')

name: صفحتنا ع الانستا طارتالي مراسلنا هناك ع حجز يراسلنا هنا ع بيج الفيسبوك حته نثبته والحاصر يعلم الغايب -  id: {'$oid': '64773fd6079df174e8aa1430'}
name: Hello everyone! How are you all? If you haven't gotten a chance, check out our Instagram! NEW and IMPROVED! https://www.instagram.com/petgearinc/ -  id: {'$oid': '64765cc29fa94c1d12f9ccfb'}
name: .                              مسابقة Qiwi Store                            منتجاتنا غير                       ومسابقاتنا غيررررر الشرووط ولا اسهل من هيك :●ا follow لصفحتنا على Facebook ●ا follow لصفحتنا على Instagram ● لايك وتعليق (ومنشن لخمس اشخاص) على البوست هاد وهيك بتكون دخلت السحب  السحب رح يكون من التعليقات  على بث مباشر على Facebook يوم الاحد 16/1/2023 الساعة 8 مساءً السحب الاول ساعة X8 ultra السحب الثاني ساعة X8 ultra  السحب الثالث aripods pro copy  السحب الرابع بورد اكرليك Qiwi store خيارك الافضل -  id: {'$oid': '6473f607a31ae693dc2de1c9'}
name: https://www.facebook.com/1000.../videos/1591429764687639/... -  id: {'$oid': '64761b5

In [30]:
torch.save(name_embeddings, 'name_embeddings_facebook.pt')
